In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
import time
import csv

# Config variables
# Define path to chrome driver
DRIVER_PATH = r"C:\Program Files (x86)\chromedriver.exe"
NUM_VIDEOS = 10
NUM_COMMENTS = 5

service = Service(executable_path=DRIVER_PATH)

# Open broswer
driver = webdriver.Chrome(service=service)

In [ ]:
# Set options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")

# Open broswer
driver = webdriver.Chrome(service=service, options=chrome_options)

In [2]:
# Enter url in the search bar
driver.get('https://www.youtube.com/')

# Expand the browser in windowed mode to full screen
driver.maximize_window()

# Find the YouTube search bar
search = WebDriverWait(driver, timeout=2).until(
    EC.presence_of_element_located((By.NAME, 'search_query'))
)

# Enter the prompt
search.send_keys("how to make a cake")
# Pressing the 'Enter' key
search.send_keys(Keys.RETURN)

# Get videos
videos = WebDriverWait(driver, timeout=2).until(
    EC.presence_of_all_elements_located((By.ID, 'video-title'))
)

# Create and prepare csv file to save information about video
with open('youtube_videos_info.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Video URL', 'Video title', 'Number of views', 'Number of likes', 'Date published', 'Comments'])

# Iterate through the videos and get URLs (requires a separate loop, due to an error)
video_urls = []

for video in videos:
    if (len(video_urls) < NUM_VIDEOS) and (video.get_attribute('href') is not None):
        video_urls.append(video.get_attribute('href'))

# Iterate through the videos and get the information we need
for url in video_urls:
    # Enter url in the search bar
    driver.get(url)

    # Get the name of the video
    title = WebDriverWait(driver, timeout=2).until(
        EC.presence_of_element_located((By.XPATH, '/html/head/meta[3]'))
    ).get_attribute('content')

    # Wait for the page to load, scroll through it, and then wait for the comments section to load
    time.sleep(3)
    driver.execute_script('window.scrollTo(0, 500);')
    time.sleep(2)

    # Get the date of publication
    try:
        date_published = WebDriverWait(driver, timeout=3).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div/meta[12]'))
        ).get_attribute('content')
    except:
            date_published = WebDriverWait(driver, timeout=3).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div/meta[9]'))
            ).get_attribute('content')

    date_published = datetime.fromisoformat(date_published)

    # Get number of views
    num_views_string = WebDriverWait(driver, timeout=3).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/div[5]/div[1]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/ytd-video-view-count-renderer/span[1]'))
    ).get_attribute('innerHTML')

    num_views = ''
    for char in num_views_string:
        if char.isdigit():
            num_views += char

    num_views = int(num_views)

    # Get number of likes
    num_likes_string = WebDriverWait(driver, timeout=3).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[2]/div[2]/div/div/ytd-menu-renderer/div[1]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button'))
    ).get_attribute('aria-label')

    num_likes = ''
    for symbol in num_likes_string:
        if symbol.isdigit():
            num_likes += symbol

    num_likes = int(num_likes)

    comments = WebDriverWait(driver, timeout=5).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content-text"]'))
    )

    # Save first N comments
    comments = [comment.text for comment in comments[:NUM_COMMENTS]]

    # Save information about video to csv file
    with open('youtube_videos_info.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([url, title, num_views, num_likes, date_published, comments])

driver.quit()

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

youtube_videos_info = pd.read_csv('youtube_videos_info.csv')

youtube_videos_info

,Video URL,Video title,Number of views,Number of likes,Date published,Comments
0,https://www.youtube.com/watch?v=EYXQmbZNhy8&pp...,The Most AMAZING Vanilla Cake Recipe,2777448,54736,2021-05-11 12:00:03-07:00,['So Happy to announce my cookbook! Pre-Order ...
1,https://www.youtube.com/watch?v=1GbMJ4Cm82o&pp...,How To Make an Eggless Cake,7208815,230366,2021-05-04 11:30:01-07:00,"[""Official HowToBasic egg plushies\n are avail..."
2,https://www.youtube.com/watch?v=qtlhdIfojmc&pp...,Classic Vanilla Cake Recipe | How to Make Birt...,12743957,161980,2020-11-05 08:00:17-08:00,['Subscribe to my channel for 3 new recipes ev...
3,https://www.youtube.com/watch?v=4M_30SNkMVg&pp...,1000+ Amazing Cake Decorating Ideas | Transfor...,8709565,36444,2022-04-19 01:30:08-07:00,['Thank you for watching !\nPlease subscribe f...
4,https://www.youtube.com/watch?v=vKTBBc13o88&pp...,cake in 2 minutes! you will make this cake eve...,4626438,40036,2022-07-17 08:49:54-07:00,"[""Thank you so much!! I have never been able t..."
5,https://www.youtube.com/watch?v=cVFePgS4eeU&pp...,Oddly Satisfying Rainbow Cake Decorating Compi...,15888534,70448,2021-03-02 06:00:30-08:00,['Thank You For Watching\nHope You Enjoy This ...
6,https://www.youtube.com/watch?v=ZjRQQFjf_VM&pp...,HI BARBIE!! Barbie: The Movie CAKE + Fluffy DI...,2261373,10791,2023-07-24 06:27:28-07:00,['Wow this cake very very nice I like ...
7,https://www.youtube.com/watch?v=hCL_N845z_Y&pp...,10000+ Perfect Cake Decorating Ideas For Every...,3639482,13541,2022-11-15 04:00:06-08:00,"['KK mesan kue 12 ya kasi bunga ya', 'Yang dik..."
8,https://www.youtube.com/watch?v=4j2PUwO_puQ&pp...,Amazing Devil's Food Cake Recipe | BEYOND Deca...,1921442,43702,2022-04-30 07:00:01-07:00,"['For chocolate cakes, think about dusting you..."
9,https://www.youtube.com/watch?v=lv9qSkv3sng&pp...,How To Make Cake Decorating Ideas | Easy Fonda...,21513943,85122,2021-12-15 04:30:11-08:00,['Thank You For Watching\nHope You Enjoy This ...
